In [29]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

gender_age_test = pd.read_csv('./input/gender_age_test.csv', index_col='device_id')
gender_age_train = pd.read_csv('./input/gender_age_train.csv', index_col='device_id')
phone_brand_device_model = pd.read_csv('./input/phone_brand_device_model.csv')
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id', keep='first').set_index('device_id')

In [38]:
phone_brand_device_model.head()

,phone_brand,device_model
device_id,,
-8890648629457979026,小米,红米
1277779817574759137,小米,MI 2
5137427614288105724,三星,Galaxy S4
3669464369358936369,SUGAR,时尚手机
-5019277647504317457,三星,Galaxy Note 2


In [7]:
phone_brand_device_model.phone_brand.value_counts().index.values

array(['小米', '三星', '华为', 'vivo', 'OPPO', '魅族', '酷派', '联想', '金立', 'HTC',
       '中兴', '乐视', '索尼', '努比亚', 'LG', '中国移动', 'TCL', '朵唯', '锤子', '优米',
       '海信', '一加', '语信', '奇酷', '天语', '摩托罗拉', '酷比', 'ZUK', '酷比魔方', '美图',
       '华硕', '艾优尼', '富可视', '梦米', '爱派尔', '黑米', '波导', '纽曼', '米歌', '诺基亚',
       '亿通', '贝尔丰', '百立丰', '欧博信', '康佳', 'LOGO', '尼比鲁', '优购', '夏新', '酷珀',
       '海尔', '维图', '聆韵', '青橙', '邦华', 'E派', 'Lovme', '至尊宝', '小杨树', '唯米',
       '糯米', '易派', '欧新', '诺亚信', '神舟', '基伍', '西米', '米奇', '沃普丰', '百加', '大可乐',
       '果米', '乡米', '斐讯', '长虹', '欧奇', '青葱', '昂达', '糖葫芦', '谷歌', '先锋', '台电',
       '欧比', 'SUGAR', '蓝魔', '奥克斯', '德赛', '白米', '大Q', '虾米', '瑞米', '金星数码',
       '首云', '飞利浦', '普耐尔', '广信', '欧沃', '欧乐迪', '摩乐', '优语', '智镁', '世纪天元',
       'PPTV', '天宏时代', '本为', '唯比', '宝捷讯', '惠普', '丰米', '鲜米', '大显', '世纪星',
       '戴尔', '恒宇丰', '凯利通', '原点', '飞秒', '赛博宇华', 'MIL', '碟米', '帷幄', '亚马逊',
       'E人E本', '宏碁', 'ZOYE', '嘉源', '西门子', '德卡诺', '欧乐酷', '极米', '瑞高'], dtype=object)

In [9]:
phone_brand_device_model.phone_brand.value_counts().index

Index(['小米', '三星', '华为', 'vivo', 'OPPO', '魅族', '酷派', '联想', '金立', 'HTC',
       ...
       '亚马逊', 'E人E本', '宏碁', 'ZOYE', '嘉源', '西门子', '德卡诺', '欧乐酷', '极米', '瑞高'],
      dtype='object', length=131)

In [10]:
phone_brand_device_model.device_model.value_counts().index

Index(['红米note', 'MI 3', 'MI 2S', 'Galaxy Note 3', 'MI 4', 'Galaxy S4',
       'Galaxy Note 2', '荣耀6', '荣耀畅玩4X', '荣耀3C',
       ...
       'Z3D 梦想板', 'Galaxy S DUOS 2', 'Z7', 'P589L', 'P21', 'V927', 'HS-U971',
       'F4', 'IUSAI US15', 'ivvi 小i Plus'],
      dtype='object', length=1599)

In [37]:
gender_age_train.head()

,gender,age,group
device_id,,,
-8076087639492063270,M,35,M32-38
-2897161552818060146,M,35,M32-38
-8260683887967679142,M,35,M32-38
-4938849341048082022,M,30,M29-31
245133531816851882,M,30,M29-31


In [66]:
phone_train = gender_age_train.merge(phone_brand_device_model, how='left', left_index=True, right_index=True)

In [82]:
phone_train.head()

,gender,age,group,phone_brand,device_model
device_id,,,,,
-8076087639492063270,M,35,M32-38,小米,MI 2
-2897161552818060146,M,35,M32-38,小米,MI 2
-8260683887967679142,M,35,M32-38,小米,MI 2
-4938849341048082022,M,30,M29-31,小米,红米note
245133531816851882,M,30,M29-31,小米,MI 3


In [83]:
brand_onehot = pd.get_dummies(phone_train.phone_brand)
model_onehot = pd.get_dummies(phone_train.device_model)

In [84]:
phone_train = phone_train.merge(brand_onehot, how='left', left_index=True, right_index=True)
phone_train = phone_train.merge(model_onehot, how='left', left_index=True, right_index=True)

In [88]:
phone_train.columns

Index(['gender', 'age', 'group', 'phone_brand', 'device_model', 'E派', 'HTC',
       'LG', 'LOGO', 'Lovme',
       ...
       '魅蓝Note3', '魅蓝metal', '魔镜 X5', '麦芒3', '麦芒3S', '麦芒4', '黄金斗士A8',
       '黄金斗士Note8', '黄金斗士S8畅玩版', '黄金斗士青春版'],
      dtype='object', length=1563)